Decoding analysis for meta-analysis map from ALE.  
Dependencies: Python 3, Neurosynth. 
Neurosynth dependencies: NumPy/SciPy,pandas,NiBabel,ply,scikit-learn.  

2019/04/23  
conducted by Chuanji Gao  

In [4]:
# At anaconda prompt, type: pip install import-ipynb
import import_ipynb
import os
os.chdir('/Users/chuanjigao/Desktop/RunningNeurosynth')
import pandas as pd

In [3]:
# Core functionality for managing and accessing data
from neurosynth import Dataset
# Analysis tools for meta-analysis, image decoding, and coactivation analysis
from neurosynth.analysis import meta, decode, network
#from neurosynth import meta, decode, network

In [4]:
dataset = Dataset.load('dataset.pkl')   # Note the capital D in the second Dataset--load() is a class method

Inputs: ALE Image (foci_av_emotionSPMfinal_ALE.nii) is an unthresholded image contains the unthresholded ALE values, one computed at every voxel in the brain. The ALE calculations first create a 3D image for each foci group using the mask, the foci and a guassian blur with a FWHM empirically derived from the subject size. These pre-ALE experiment-level images are called Modeled Activation (MA) maps. The MA maps can be calculated by finding the union or the maximum across each focus's Gaussian. Using the maximum limits the effect of an experiment with multiple foci very near one another and is referred to as "non-additive" in the preferences. The ALE image is a union of all of the MA maps.  

The decoding wil not work unless the input image has the same dimensions as the typicall MNI152 2mm image.  

In [5]:
# Decode images: simple try
decoder = decode.Decoder(dataset, features=['emotion', 'auditory', 'visual', 'audiovisual'])
data = decoder.decode(['foci_av_emotionSPMfinal_ALE_resliced.nii'], save='decoding_results_ALE_simple.txt')

/anaconda3/lib/python3.7/site-packages/neurosynth/analysis/meta.py:134: RuntimeWarning: divide by zero encountered in true_divide
  pFgA = pAgF * pF / pA
/anaconda3/lib/python3.7/site-packages/neurosynth/analysis/meta.py:134: RuntimeWarning: invalid value encountered in true_divide
  pFgA = pAgF * pF / pA
/anaconda3/lib/python3.7/site-packages/neurosynth/analysis/meta.py:139: RuntimeWarning: divide by zero encountered in true_divide
  pFgA_prior = pAgF * prior / pAgF_prior
/anaconda3/lib/python3.7/site-packages/neurosynth/analysis/meta.py:139: RuntimeWarning: invalid value encountered in true_divide
  pFgA_prior = pAgF * prior / pAgF_prior


Understanding term similarity score in Decoder:  
The decoder returns map-wise correlation coefficients between the input and the reverse inference maps.  
In other words, it's just pearson correlation between the two vectorized maps, computed over all voxels. I.e., corr(x_1, x_2),where x_1 and x_2 are aligned vectors of voxels values from each of the two maps.  

How are voxels with missing values handled?  
Attempting to decode maps with relatively few non-zero values (those conservatively corrected for multiple comparisions) will produce biased results (i.e., many coefficients very close to 0). Note that the deliberate introduction of bias is not necessarily a bad thing here, because the laternative is to produce highly variable estimates that will often provide a misleading sense of the robustness of an association. In future, we will provide a user option for handing of 0 values. In general, however, we recommend decoding unthresholded, uncorrected, whole-brain maps whenever possible.  

The conclusion needs to be made with caution: "e.g., there is some evidence that our pattern of activation is more consistent with language and motor processes than other kinds of processes".  

Note that you probably want to pay attention to the absolute strength of the correlations too, as that can give you an informative sense of how "typically" you results are. It can be interpreted exactly like any other correlation coefficient. Which is to say, they have a clear statistical meaning (1 is identity, -1 is inversion, 0 is independence). E.g., if the single strongest correlation between your map and any of the reverse inference meta-analyses is only, say .06, the implication is probably a) your analysis is underpowered b) you have a task that is tapping some uncommon combination of cognitive processes. However, the implications for any particular application depend entirely on the context. There is no simple answer to whether a value of say, .2 is large or small.  

We can't know whether the association between the map input and the neurosynth meta-analysis map is significant or not.  

In [6]:
# Decode images: all features, 
# if we left the features argument unspecified, the decoder would default to using the entire set of 500+ features
decoder = decode.Decoder(dataset)
data = decoder.decode(['foci_av_emotionSPMfinal_ALE_resliced.nii'], save='decoding_results_ALE_All_Features.txt')

In [5]:
data = pd.read_csv('decoding_results_ALE_All_Features.txt', sep=" ", header=None)
data.columns = ["Features", "r"]

ParserError: Error tokenizing data. C error: Expected 1 fields in line 5, saw 2
